In [9]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb

from sklearn.decomposition import PCA


## (the following code is same as s4_models_classification)

In [10]:
dfcombo = pd.read_csv('combo.csv')
# dfcombo.describe().T

In [11]:
years_to_encode = [2019, 2020, 2021]
for year in years_to_encode:
    dfcombo[f'year_{year}'] = (dfcombo['year'] == year).astype(int)

In [12]:
dfcombo['year'] = dfcombo['year'].astype('category')

In [13]:
featuresX = dfcombo.columns.tolist()
del featuresX[31:]
del featuresX[0:2]
featuresA = featuresX + ['year_2019', 'year_2020', 'year_2021']
featuresB = featuresX + ['year']
ycol='pct_next_1yr'
y = [ycol]
# featuresA

In [14]:
dfcombo['weighted_avg'] = (dfcombo['population'] * dfcombo['pct_next_1yr']).groupby(dfcombo['year']).transform('sum') / dfcombo['population'].groupby(dfcombo['year']).transform('sum')
dfcombo['winner'] = dfcombo.apply(lambda x: 1 if x['pct_next_1yr'] > x['weighted_avg'] else 0, axis=1)

## PCA

In [37]:
X = dfcombo[featuresB]
y = dfcombo['winner']
# no need for StandardScaler
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

pca = PCA(n_components=13)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

## Random Forest Classifier with PCA

In [39]:
rf_clf = RandomForestClassifier(random_state=42)
rf_clf.fit(X_train_pca, y_train)
y_pred = rf_clf.predict(X_test_pca)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy*100:.2f}%")

Accuracy: 70.87%


In [ ]:
# 2:  55
# 5:  62.46
# 10: 70.50
# 12: 70.82
# 13: 70.87  <- best. from 29 features to 13, accuracy only decreased from 73.34 (63 sec) to 70.87 (46 sec)
# 14: 70.78
# 15: 70.80
# 22: 70.54
# 30: 70.41
